# Performance Analysis -  Numpy
> Number of effective sequences implemented in Numpy
- toc: true
- branch: master
- badges: true
- author: Donatas Repečka
- categories: [performance]

## Introduction

In [the previous post](https://donatasrep.github.io/donatas.repecka/performance/2021/04/27/Performance-comparison.html) I have compared various languages and libraries in terms of their speed. This notebook contains the code used in the comparison as well as some details about the choices made to improve the performance of numpy implementation.

## Setup

In [1]:
# !wget https://github.com/donatasrep/donatas.repecka/blob/master/data/picked_msa.fasta

In [2]:
# ! pip install numpy
# ! pip install pandas

## Getting data

I will cheat here and use pandas to help me to read the file. 

In [1]:
import pandas as pd

In [2]:
def get_data(path):
    fasta_df = pd.read_csv(path, sep="\n", lineterminator=">", index_col=False, names=['id', 'seq'])
    return fasta_df.seq.to_numpy(dtype=str)

In [3]:
seqs = get_data('../data/picked_msa.fasta')

## Numpy implementation

Just to remind the pseudo code looks like this:

```
for seq1 in seqs:
  for seq2 in seqs:
    if count_mathes(seq1, seq2) > threshold:
      weight +=1
  meff += 1/weight
 
meff = meff/(len(seq1)^0.5)
```

This Numpy implementation is based on pure python implementation which can be found [here](https://donatasrep.github.io/donatas.repecka/performance/2021/05/08/performance-pure-python.html). 

The main differences are:
* uses numpy arrays and operators. 
* sequences are in vectorised fashion using `np.equal` rather than looping over each element in the sequence.
* sequences are converted to arrays of integers rather than characters.


In [4]:
import numpy as np
def get_nf_python_numpy(seqs, threshold=0.8):
    seqs = seqs.view(np.uint32).reshape(seqs.shape[0], -1)
    n_seqs = len(seqs)
    is_same_cluster = np.ones([n_seqs, n_seqs], np.bool_) 
    for i in range(n_seqs):
        for j in range(i+1, n_seqs):
            identity = np.equal(seqs[i], seqs[j]).mean()
            is_more = np.greater(identity, threshold)
            is_same_cluster[i,j] = is_more
            is_same_cluster[j,i] = is_more
    meff = 1.0/np.sum(is_same_cluster,1)
    return meff.sum()/(seqs.shape[-1]**0.5)

In [5]:
%%timeit -n 3 -r 3
get_nf_python_numpy(seqs[:100])

141 ms ± 2.95 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [6]:
get_nf_python_numpy(seqs[:100])

0.18006706787628665

Although performance gain is substantial, I still have two loops which is one of the first things you want to tackle when trying to improve algorithm speed. In practise, you want to use vectorization to take advantage of CPU ability to perform the same operation on vectors. 

Ideally, I would like to do all to all pairwise comparison in one vectorized operation. Unfortunately, this is not a viable option due to memory constraints . Such operation will require `n * n * l` size matrix. In my case, that  would result into ~70GB size matrix (n = 10000, l = 683, boolean uses 1 byte)

$$10000^2 *683 = 68.3*10^9 \ bytes = 68.3 \ gigabytes$$

To be able to run such an algorithm on an everyday computer, the memory requirement has to be lower. Usually, this can be solved with batching, taking only a portion of data at the time. In this case, I will perform a one-to-all comparison at the time. This will require `n*l` memory which is 10000 (~7MB)  times less than for the all-to-all version. 


Note, I still can exploit the fact that the matrix of identity is symmetric which means that only the first iteration has to be done with `n-1` sequences, the second sequence needs only be compared with `n-2` and so on. 

In [5]:
def get_one_to_all_comparison(seqs, threshold=0.8):
    pairwise_id = np.equal(seqs[1:], seqs[0].T).mean(1)
    return pairwise_id > threshold

In [6]:
def get_nf_numpy_v2(seqs):
    seqs = seqs.view(np.uint32).reshape(seqs.shape[0], -1)
    n_seqs, seq_len = seqs.shape
    is_same_cluster = np.ones([n_seqs, n_seqs],np.bool_)
    for i in range(n_seqs-1):
        out = get_one_to_all_comparison(seqs[i:])
        is_same_cluster[i, i+1:] = out
        is_same_cluster[i+1:, i] = out
    s = 1.0/is_same_cluster.sum(1)
    return s.sum()/(seq_len**0.5)

In [9]:
%%timeit -n 3 -r 3
get_nf_numpy_v2(seqs[:100])

14.8 ms ± 2.97 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


By using numpy, the algorithm's runtime decreases significantly. The gain on 1% of the data is roughly 100 times (might differ depending on the hardware). The improvement will scale with the amount of data that is being processed so on the data set  the gain will be even larger. 


## Multi threading

There is a limit to how much an algorithm can be improved. However, more speed can be gained by simply exploiting the sheer size of infrastructure, i.e. multiple CPUs/machines. Unfortunately, this rarely happens automatically, unless you use libraries that have this ability built-in. Although Numpy has some functionality that runs multi-threaded, in this case some additional work is required. 

In this case, sequence comparison does not need to be performed sequentially, thus can be parallelized.

In [7]:
import multiprocessing
from multiprocessing import sharedctypes

In [9]:
seqs = seqs[:100].view(np.uint32).reshape(100, -1)
n_seqs, seq_len = seqs.shape

global_is_same_cluster = np.ctypeslib.as_ctypes(np.ones([n_seqs, n_seqs],np.bool_))
shared_global_is_same_cluster = sharedctypes.RawArray(global_is_same_cluster._type_, global_is_same_cluster)

def get_nf_numpy_global(i, threshold=0.8):
    out = get_one_to_all_comparison(seqs[i:])
    tmp = np.ctypeslib.as_array(shared_global_is_same_cluster)
    tmp[i, i+1:] = out
    tmp[i+1:, i] = out

def get_nf_multi_threaded(threads=None):
    with multiprocessing.Pool(threads) as pool:
        pool.map(get_nf_numpy_global, range(n_seqs))
    global_is_same_cluster = np.ctypeslib.as_array(shared_global_is_same_cluster)
    meff = 1.0/global_is_same_cluster.sum(1)
    return meff.sum()/(seq_len**0.5)

This is not the nicest solution, I have to admit, but that still demonstrates the power of having multithreading enabled.

Note, IPython on Windows struggles with multiprocessing and this code will not work [IPython on Windows struggles with multiprocessing and this code will not work](https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac). 
